<a href="https://colab.research.google.com/github/shige-ta/WeatherBinaryClassifier/blob/main/classify_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: data/
   creating: data/not_sunny/
  inflating: data/not_sunny/00202-2349356982.png  
  inflating: data/not_sunny/00203-2349356982.png  
  inflating: data/not_sunny/00204-2349356983.png  
  inflating: data/not_sunny/00205-2349356984.png  
  inflating: data/not_sunny/00206-2349356985.png  
  inflating: data/not_sunny/00207-2349356986.png  
  inflating: data/not_sunny/00208-2349356987.png  
  inflating: data/not_sunny/00209-2349356988.png  
  inflating: data/not_sunny/00210-2349356989.png  
  inflating: data/not_sunny/00211-2349356990.png  
  inflating: data/not_sunny/00212-2349356991.png  
  inflating: data/not_sunny/00213-2349356992.png  
  inflating: data/not_sunny/00214-2349356993.png  
  inflating: data/not_sunny/00215-2349356994.png  
  inflating: data/not_sunny/00216-2349356995.png  
  inflating: data/not_sunny/00217-2349356996.png  
  inflating: data/not_sunny/00218-2349356997.png  
  inflating: data/not_sunny/00219-2349356998.png  
  infl

In [7]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import random_split

# データセットのパスを指定
data_dir = "/content/data"

# 前処理を定義
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 画像データセットを読み込みます
image_datasets = datasets.ImageFolder(data_dir, transform=data_transforms)

# データセットを訓練用と検証用に分割します
train_size = int(0.8 * len(image_datasets))
val_size = len(image_datasets) - train_size
train_dataset, val_dataset = random_split(image_datasets, [train_size, val_size])

# DataLoaderを作成
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

# 事前訓練済みのResNetモデルをロード
model = models.resnet50(pretrained=True)

# 最終層を変更
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# 損失関数と最適化手法を設定
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# モデルを訓練
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# モデルを保存
torch.save(model.state_dict(), "/content/model.pth")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 122MB/s]


In [11]:
import torch
from torchvision import transforms, models
from PIL import Image

# 学習済みモデルのパス
model_path = '/content/model.pth'

# 画像のパス
# image_path = '/content/sunny.png'

image_path = '/content/not_sunny.png'

# 前処理を定義
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# モデルをロード
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.load_state_dict(torch.load(model_path))
model.eval()

# 画像を読み込み、前処理を適用
image = Image.open(image_path)

# RGBA画像をRGBに変換
if image.mode == 'RGBA':
    image = image.convert('RGB')
image = transform(image).unsqueeze(0)

# モデルを使って画像を予測
output = model(image)

# ソフトマックス関数を適用して確率を計算
probabilities = torch.nn.functional.softmax(output, dim=1)
print('Probabilities:', probabilities.detach().numpy())

# 予測結果を解釈
_, predicted = torch.max(output, 1)
if predicted.item() == 0:
    print('The image is predicted to be sunny.')
else:
    print('The image is predicted to be not sunny.')


Probabilities: [[0.5307115  0.46928856]]
The image is predicted to be sunny.


In [12]:
#データの追加が必要？